In [10]:
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import numpy as np

In [11]:
def compareClassifiers(train_data, test_data):
    clfByaes = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf', MultinomialNB(alpha=1e-3, fit_prior=False)),
        ])
    clfByaes.fit(train_data.data,train_data.target)
    predictedBayes = clfByaes.predict(test_data.data)
    print(np.mean(predictedBayes == test_data.target))

    clfSGD = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                alpha=1e-3, random_state=42,
                                max_iter=5, tol=None)),
        ])
    clfSGD.fit(train_data.data,train_data.target)
    predictedSGD = clfSGD.predict(test_data.data)
    print(np.mean(predictedSGD == test_data.target))

    clfBernoulliNB = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf', RidgeClassifier()),
        ])
    clfBernoulliNB.fit(train_data.data,train_data.target)
    predictedBernoulliNB = clfBernoulliNB.predict(test_data.data)
    print(np.mean(predictedBayes == test_data.target))

In [12]:
# Categories
categories = ['comp.graphics','comp.os.ms-windows.misc','comp.sys.ibm.pc.hardware','comp.sys.mac.hardware','comp.windows.x',
              'rec.autos','rec.motorcycles','rec.sport.baseball','rec.sport.hockey',
              'sci.crypt','sci.electronics','sci.med','sci.space',
              'misc.forsale','talk.politics.misc','talk.politics.guns','talk.politics.mideast',
              'talk.religion.misc','alt.atheism','soc.religion.christian']

# Load train and test data 
twenty_train = fetch_20newsgroups(subset='train',
    categories=categories, shuffle=True, random_state=42)
twenty_test = fetch_20newsgroups(subset='test',
    categories=categories, shuffle=True, random_state=42)

In [13]:
# print(twenty_test.data)


compareClassifiers(twenty_train,twenty_test)

# for doc, category in zip(docs_new, predicted):
#     print('%r => %s' % (doc, twenty_train.target_names[category]))


# from sklearn import metrics
# print(metrics.classification_report(twenty_test.target, predicted,
#     target_names=twenty_test.target_names))

# print(metrics.confusion_matrix(twenty_test.target, predicted))

# from sklearn.model_selection import GridSearchCV
# parameters = {
#     'vect__ngram_range': [(1, 1), (1, 2)],
#     'tfidf__use_idf': (True, False),
#     'clf__alpha': (1e-2, 1e-3),
# }

# gs_clf = GridSearchCV(text_clf, parameters, cv=5, n_jobs=-1)

# gs_clf = gs_clf.fit(twenty_train.data[:400], twenty_train.target[:400])

# twenty_train.target_names[gs_clf.predict(['God is love'])[0]]

# print(gs_clf.best_score_)

# for param_name in sorted(parameters.keys()):
#     print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

# print(gs_clf.cv_results_)


0.7738980350504514
0.8248805098247477
0.7738980350504514
